# STAC API Functions

Functions for talking to a stac-server implementation.

In [1]:
#! pip install geopandas

In [2]:
from functools import lru_cache
import time
from typing import Union

import requests

_stac = 'https://ibhoyw8md9.execute-api.us-west-2.amazonaws.com/prod'

def get_json(url: str, params: Union[list, dict, None] = None) -> Union[list, dict]:
    """
    Perform a GET request to a REST API endpoint, assume the response is a JSON
    """
    resp = requests.get(url, params=params, timeout=5)
    
    if not resp.ok:
        resp.raise_for_status()

    return resp.json()

def post_json(url: str, data: Union[list, dict]) -> Union[list, dict]:
    """
    Send a POST request to the resource with the payload as JSON
    """
    resp = requests.post(url, json=data, timeout=5)
    
    if not resp.ok:
        resp.raise_for_status()
        
    return resp.json()

@lru_cache()
def search_endpoint(endpoint: str = _stac) -> str:
    """
    Get the search endpoint for STAC
    """
    resp = get_json(endpoint)
    
    for item in resp['links']:
        if item['rel'] == 'search':
            return item['href']
        
def get_next(resp: dict) -> Union[str, None]:
    for link in resp['links']:
        if link['rel'] == 'next':
            return link['body']

def stac_query(query_params: dict, endpoint: str = _stac) -> Union[list, dict]:
    """
    Perform query against the STAC API, and iterate through the pages of results to grab everything
    
    All we really care about are the 'features' that get returned from the query results
    """
    resp = post_json(search_endpoint(endpoint), data=query_params)
#     return resp
    feat = resp['features']
    nxt = get_next(resp)

    if nxt:
        feat.extend(stac_query(nxt, endpoint))

    return feat

# Geospatial Functions

Functions for generating boundaries in the ARD CRS.

In [3]:
from typing import Tuple
import geopandas as gpd
from shapely.geometry import Polygon

def regional_affine(region: str) -> Tuple[int, int, int, int, int, int]:
    """
    Return the various regional ARD GeoTransforms
    """
    if region == 'cu':
        return -2565585, 150000, 0, 3314805, 0, -150000
    elif region == 'ak':
        return -851715, 150000, 0, 2474325, 0, -150000
    elif region == 'hi':
        return -444345, 150000, 0, 2168895, 0, -150000
    else:
        raise ValueError('Unrecognized region')
        
def regional_wkt(region: str) -> str:
    """
    Return the various regional ARD projection information
    """
    if region == 'cu':
        return 'PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.2569999999957,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'
    elif region == 'ak':
        return 'PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.2569999999986,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",55],PARAMETER["standard_parallel_2",65],PARAMETER["latitude_of_center",50],PARAMETER["longitude_of_center",-154],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'
    elif region == 'hi':
        return 'PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.2569999999986,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",8],PARAMETER["standard_parallel_2",18],PARAMETER["latitude_of_center",3],PARAMETER["longitude_of_center",-157],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'
    else:
        raise ValueError('Unrecognized region')

def tile_ul(h: int, v: int, region: str) -> Tuple[int, int]:
    """
    Determine the upper-left x/y for an ARD tile.
    """
    affine = regional_affine(region)
    xmin = affine[0] + h * 5000 * 30
    ymax = affine[3] - v * 5000 * 30

    return xmin, ymax

def tile_lr(h: int, v: int, region: str) -> Tuple[int, int]:
    """
    Determine the lower-right x/y for an ARD tile.
    """
    return tile_ul(h + 1, v + 1, region)

def tile_ur(h: int, v: int, region: str) -> Tuple[int, int]:
    """
    Determine the upper-right x/y for an ARD tile.
    """
    return tile_ul(h + 1, v, region)

def tile_ll(h: int, v: int, region: str) -> Tuple[int, int]:
    """
    Determine the lower-left x/y for an ARD tile.
    """
    return tile_ul(h, v + 1, region)

def tile_bounds(h: int, v: int, region: str) -> gpd.GeoDataFrame:
    """
    Build a GeoDataFrame object for a tile
    """
    return gpd.GeoDataFrame([{'h': h,
                              'v': v,
                              'geometry': Polygon([tile_ul(h, v, region),
                                                   tile_ur(h, v, region),
                                                   tile_lr(h, v, region),
                                                   tile_ll(h, v, region)])}]).set_crs(regional_wkt(region))

In [4]:
def query_tile(h: int, v: int, region: str):
    """
    Example function for querying the stac-server for all the c2l2-sr scenes that intersect a given tile.
    """
    query = {'bbox': f"[{','.join(str(_) for _ in tile_bounds(h, v, region).to_crs('EPSG:4326')['geometry'][0].bounds)}]",
             'limit': 300,
             'datetime': '1984-01-01/2020-12-31',
             'collections': ['landsat-c2l2-sr']}
    
    return stac_query(query)

In [5]:
h, v = 3, 10
region = 'cu'

t1 = time.time()
items = query_tile(h, v, region)
print(time.time() - t1)

36.170605421066284


In [6]:
len(items)

7269

In [7]:
dir(items)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [8]:
for item in items:
    print(item)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'type': 'Feature', 'stac_version': '1.0.0', 'stac_extensions': ['https://landsat.usgs.gov/stac/landsat-extension/v1.1.1/schema.json', 'https://stac-extensions.github.io/view/v1.0.0/schema.json', 'https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/file/v1.0.0/schema.json', 'https://stac-extensions.github.io/eo/v1.0.0/schema.json', 'https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json', 'https://stac-extensions.github.io/storage/v1.0.0/schema.json'], 'id': 'LE07_L2SP_041035_20160223_20200903_02_T1_SR', 'description': 'Landsat Collection 2 Level-2 Surface Reflectance Product', 'bbox': [-119.15317583314766, 35.09130229317918, -116.5744722302763, 36.99299292146264], 'geometry': {'type': 'Polygon', 'coordinates': [[[-118.70270380086754, 36.99299292146264], [-119.15317583314766, 35.402146044139535], [-117.06242444859552, 35.09130229317918], [-116.5744722302763, 36.67991864125531], [-118.70270380086754, 36.99299292146264]]]}, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




{'type': 'Feature', 'stac_version': '1.0.0', 'stac_extensions': ['https://landsat.usgs.gov/stac/landsat-extension/v1.1.1/schema.json', 'https://stac-extensions.github.io/view/v1.0.0/schema.json', 'https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/file/v1.0.0/schema.json', 'https://stac-extensions.github.io/eo/v1.0.0/schema.json', 'https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json', 'https://stac-extensions.github.io/storage/v1.0.0/schema.json'], 'id': 'LE07_L2SP_042035_20010103_20200917_02_T1_SR', 'description': 'Landsat Collection 2 Level-2 Surface Reflectance Product', 'bbox': [-120.69313796561175, 35.09060919164476, -118.20327577956482, 37.00027244497288], 'geometry': {'type': 'Polygon', 'coordinates': [[[-120.24601976876485, 37.00027244497288], [-120.69313796561175, 35.38161390206052], [-118.69150558921754, 35.09060919164476], [-118.20327577956482, 36.70299454866657], [-120.24601976876485, 37.00027244497288]]]}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
print((items[0].keys()))

dict_keys(['type', 'stac_version', 'stac_extensions', 'id', 'description', 'bbox', 'geometry', 'properties', 'assets', 'links', 'collection'])


In [10]:
print(items[0]['assets']['reduced_resolution_browse']['alternate']['s3']['href'])

s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/041/036/LC08_L2SP_041036_20201226_20210310_02_T1/LC08_L2SP_041036_20201226_20210310_02_T1_thumb_large.jpeg


In [11]:
print(items[0]['assets'].keys())

dict_keys(['thumbnail', 'reduced_resolution_browse', 'index', 'coastal', 'blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'qa_aerosol', 'ANG.txt', 'MTL.txt', 'MTL.xml', 'MTL.json', 'qa_pixel', 'qa_radsat'])


In [12]:
print(items[0]['assets']['red']['alternate']['s3']['href'])

s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/041/036/LC08_L2SP_041036_20201226_20210310_02_T1/LC08_L2SP_041036_20201226_20210310_02_T1_SR_B4.TIF


# the word standard may mean UTM - if I remember Pete Doucette's definition